In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from FinancialDataAPI import FinancialDataAPI
from datetime import date, datetime, timedelta
import pandas as pd
import numpy as np

In [3]:
api = FinancialDataAPI()

In [4]:
ref_date = date(2018, 12, 31)
fwd_date = date(2019, 1, 31)

In [5]:
universe = api.get_all_tickers(as_of_date=ref_date)
len(universe)

2359

In [6]:
api.display_field_info('Volume')

Volume (Volume) 
  Parameters: start, end, fill_prev 
  start: Date, end: Date, fill_prev: Str [y/n] 




In [7]:
interval = timedelta(days=30)

volume_30d = api.get_data(
    universe, 'Volume', 
    start=ref_date - interval, end=ref_date, fill_prev='y', adj='n'
)

price_30d = api.get_data(
    universe, 'Close', 
    start=ref_date - interval, end=ref_date, fill_prev='y', adj='n'
)

price_30d['Turnover'] = volume_30d['Volume'] * price_30d['Close']

price_30d.head()

,Date,Close,Turnover
Ticker,,,
GOOG,2018-12-01,1094.43,2.824299e+09
GOOG,2018-12-02,1094.43,2.824299e+09
GOOG,2018-12-03,1106.43,2.202634e+09
GOOG,2018-12-04,1050.82,2.464347e+09
GOOG,2018-12-05,1050.82,2.464347e+09


In [8]:
avg_turnover_30d = price_30d.groupby('Ticker')[['Turnover']].mean()

avg_turnover_30d.head()

,Turnover
Ticker,
A,1.823797e+08
AA,9.521177e+07
AAL,3.102825e+08
AAMC,2.526800e+05
AAME,6.597855e+03


In [9]:
tradable_universe = avg_turnover_30d[avg_turnover_30d['Turnover'] >= 50 * 10 ** 6].index.tolist()
len(tradable_universe)

790

In [15]:
price = api.get_data(
    tradable_universe, 'Close', start=ref_date, end=fwd_date, 
    adj='y', fill_prev='y'
)

In [ ]:
sh_out = api.get_data(
    tradable_universe, 'Shares Outstanding', 
    start=ref_date, end=ref_date, fill_prev='y'
)

price_unadjusted = api.get_data(
    tradable_universe, 'Close', start=ref_date, end=ref_date, 
    adj='n', fill_prev='y'
)

price_unadjusted['Mkt Cap'] = sh_out['Shares Outstanding'] * price_unadjusted['Close']
price_unadjusted

In [ ]:
price = api.get_data(
    tradable_universe, 'Close', start=ref_date, end=ref_date, 
    adj='y', fill_prev='y'
)

price_fwd = api.get_data(
    tradable_universe, 'Close', start=fwd_date, end=fwd_date, 
    adj='y', fill_prev='y'
)

price['Forward Return'] = (price_fwd['Close'] / price['Close'] - 1) * 100

price

In [ ]:
price['Mkt Cap'] = price_unadjusted['Mkt Cap']
price

In [ ]:
interval = timedelta(days=365)

price_1y = api.get_data(
    large_cap_tickers, 'Close', start=ref_date - interval, end=ref_date - interval, 
    adj='y', fill_prev='y'
)

price['Momentum'] = (price['Close'] / price_1y['Close'] - 1) * 100
price

In [ ]:
len(price.dropna())

In [ ]:
price = price.dropna().copy()

In [ ]:
num_q = 5
price['Mkt Cap Q'] = pd.qcut(price['Mkt Cap'], num_q, labels=range(1, num_q+1))
price['Momentum Q'] = pd.qcut(price['Momentum'], num_q, labels=range(1, num_q+1))
price

In [ ]:
price.groupby('Mkt Cap Q')[['Forward Return']].mean()

In [ ]:
price.groupby('Momentum Q')[['Forward Return']].mean()

In [ ]:
net_income = api.get_data(
    universe, 'Net Income', pt='ttm', offset_start=0, offset_end=0, 
    as_of_date_start=ref_date, as_of_date_end=ref_date
)

sh_out = api.get_data(
    universe, 'Shares Outstanding', 
    start=ref_date, end=ref_date, fill_prev='y'
)

net_income['EPS'] = net_income['Net Income'] / sh_out['Shares Outstanding']
net_income

In [ ]:
net_income['EPS Q'] = pd.qcut(net_income['EPS'], num_q, labels=range(1, num_q+1))
net_income['Forward Return'] = price['Forward Return']
net_income[net_income['EPS'] > 0].groupby('EPS Q')[['Forward Return']].mean()